In [1]:
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import time

import numpy as np
import pandas as pd
import requests

In [3]:
!mkdir -p ../data/DataSet_APPA/raw

Posizione delle stazioni: https://www.appa.provincia.tn.it/Documenti-e-dati/Risorse/Rete-di-monitoraggio-qualita-dell-aria-in-Trentino

In [26]:
data_folder = "../data/DataSet_APPA/raw"


In [27]:
basepath = "https://bollettino.appa.tn.it/aria/opendata/csv"
variables = "2,4,6,8,9,15,22,23"

for year in range(2023, 2024):
    print(year)
    for month in range(1, 13):
        print(".", end="")
        end_month = month + 1

        if end_month <= 12:
            end_year = year
        else:
            end_month = 1
            end_year = year + 1

        url = f"{basepath}/{year}-{month:02d}-01,{end_year}-{end_month:02d}-01/{variables}"

        time.sleep(0.1)  # sleep 100 ms to avoid being blocked
        response = requests.get(url)
        if response.status_code == 200:
            filename = os.path.join(data_folder, f"{year}-{month}.csv")

            with open(filename, "wb") as file:
                file.write(response.content)
        else:
            print(url)
            print(response.status_code)

    print("")

2023
............


In [28]:
df = []

for path in os.listdir(data_folder):
    fullpath = os.path.join(data_folder, path)
    df.append(
        read_csv(
            fullpath,
            sep=",",
            index_col=False,
            encoding="latin-1",
        )
    )

df = pd.concat(df)

In [29]:
appa_data = df.copy()

In [30]:
# APPA_Data = APPA_Data.drop("Unità di misura",axis = 1)
appa_data = appa_data.dropna()
appa_data.drop_duplicates(inplace=True)
appa_data

,Stazione,Inquinante,Data,Ora,Valore,Unità di misura
0,Parco S. Chiara,PM10,2018-12-01,1,52,µg/mc
1,Parco S. Chiara,PM10,2018-12-01,2,143,µg/mc
2,Parco S. Chiara,PM10,2018-12-01,3,76,µg/mc
3,Parco S. Chiara,PM10,2018-12-01,4,44,µg/mc
4,Parco S. Chiara,PM10,2018-12-01,5,42,µg/mc
...,...,...,...,...,...,...
20748,A22 (Avio),Ossido di Carbonio,2013-07-31,20,0.5,mg/mc
20749,A22 (Avio),Ossido di Carbonio,2013-07-31,21,0.6,mg/mc
20750,A22 (Avio),Ossido di Carbonio,2013-07-31,22,0.5,mg/mc
20751,A22 (Avio),Ossido di Carbonio,2013-07-31,23,0.6,mg/mc


In [31]:
appa_data["Data"] = pd.to_datetime(appa_data["Data"], format="%Y-%m-%d")
appa_data["Time"] = appa_data["Data"] + pd.to_timedelta(appa_data["Ora"], unit="h")
appa_data = appa_data.drop(columns=["Data", "Ora"])
appa_data.rename(columns={"Time": "Data"}, inplace=True)

appa_data

,Stazione,Inquinante,Valore,Unità di misura,Data
0,Parco S. Chiara,PM10,52,µg/mc,2018-12-01 01:00:00
1,Parco S. Chiara,PM10,143,µg/mc,2018-12-01 02:00:00
2,Parco S. Chiara,PM10,76,µg/mc,2018-12-01 03:00:00
3,Parco S. Chiara,PM10,44,µg/mc,2018-12-01 04:00:00
4,Parco S. Chiara,PM10,42,µg/mc,2018-12-01 05:00:00
...,...,...,...,...,...
20748,A22 (Avio),Ossido di Carbonio,0.5,mg/mc,2013-07-31 20:00:00
20749,A22 (Avio),Ossido di Carbonio,0.6,mg/mc,2013-07-31 21:00:00
20750,A22 (Avio),Ossido di Carbonio,0.5,mg/mc,2013-07-31 22:00:00
20751,A22 (Avio),Ossido di Carbonio,0.6,mg/mc,2013-07-31 23:00:00


In [32]:
appa_data["Valore"] = appa_data.Valore.map(lambda x: np.nan if x == "n.d." else float(x))
appa_data

,Stazione,Inquinante,Valore,Unità di misura,Data
0,Parco S. Chiara,PM10,52.0,µg/mc,2018-12-01 01:00:00
1,Parco S. Chiara,PM10,143.0,µg/mc,2018-12-01 02:00:00
2,Parco S. Chiara,PM10,76.0,µg/mc,2018-12-01 03:00:00
3,Parco S. Chiara,PM10,44.0,µg/mc,2018-12-01 04:00:00
4,Parco S. Chiara,PM10,42.0,µg/mc,2018-12-01 05:00:00
...,...,...,...,...,...
20748,A22 (Avio),Ossido di Carbonio,0.5,mg/mc,2013-07-31 20:00:00
20749,A22 (Avio),Ossido di Carbonio,0.6,mg/mc,2013-07-31 21:00:00
20750,A22 (Avio),Ossido di Carbonio,0.5,mg/mc,2013-07-31 22:00:00
20751,A22 (Avio),Ossido di Carbonio,0.6,mg/mc,2013-07-31 23:00:00


In [33]:
appa_data = appa_data.set_index("Data")  # Set "Data" as the new index
appa_data

,Stazione,Inquinante,Valore,Unità di misura
Data,,,,
2018-12-01 01:00:00,Parco S. Chiara,PM10,52.0,µg/mc
2018-12-01 02:00:00,Parco S. Chiara,PM10,143.0,µg/mc
2018-12-01 03:00:00,Parco S. Chiara,PM10,76.0,µg/mc
2018-12-01 04:00:00,Parco S. Chiara,PM10,44.0,µg/mc
2018-12-01 05:00:00,Parco S. Chiara,PM10,42.0,µg/mc
...,...,...,...,...
2013-07-31 20:00:00,A22 (Avio),Ossido di Carbonio,0.5,mg/mc
2013-07-31 21:00:00,A22 (Avio),Ossido di Carbonio,0.6,mg/mc
2013-07-31 22:00:00,A22 (Avio),Ossido di Carbonio,0.5,mg/mc


In [34]:
appa_data.Stazione.value_counts()

Stazione
Parco S. Chiara    571282
Rovereto           453390
Borgo Valsugana    447300
A22 (Avio)         398388
Riva del Garda     343214
Via Bolzano        342691
Piana Rotaliana    337652
Monte Gaza         284321
Name: count, dtype: int64

In [35]:
appa_data.Inquinante.value_counts()

Inquinante
Biossido di Azoto     915441
PM10                  904969
Ozono                 666598
PM2.5                 328642
Ossido di Carbonio    248313
Biossido Zolfo        114275
Name: count, dtype: int64

In [36]:
!mkdir -p ../data/DataSet_APPA/processed/

In [38]:
appa_data.to_csv("../data/DataSet_APPA/processed/appa_data.csv")